In [14]:
import dai
import pandas as pd

In [15]:
df = dai.query("SELECT DISTINCT date FROM all_trading_days WHERE market_code = 'CN'").df()
df = df.sort_values("date").reset_index(drop=True)
df

,date
0,1990-12-03
1,1990-12-04
2,1990-12-05
3,1990-12-06
4,1990-12-07
...,...
8804,2026-12-25
8805,2026-12-28
8806,2026-12-29
8807,2026-12-30


In [16]:
df["dt_year"]      = df["date"].dt.year.astype(str)
df["dt_quarter"]   = df["date"].dt.to_period("Q").apply(lambda x: f"{x.year}{x.quarter:02}")
df["dt_month"]     = df["date"].dt.to_period("M").apply(lambda x: f"{x.year}{x.month:02}")
df["dt_fortnight"] = df["date"].apply(lambda x: f"{x.year}{((x.dayofyear - 1) // 14) + 1:02}")
df["dt_week"]      = df["date"].dt.strftime('%Y%U')
df["dt_day"]       = df["date"].dt.strftime("%Y%m%d")

df["is_year_start_trade"]      = (df["dt_year"] != df["dt_year"].shift(1)).astype(int)
df["is_year_end_trade"]        = (df["dt_year"] != df["dt_year"].shift(-1)).astype(int)
df["is_quarter_start_trade"]   = (df["dt_quarter"] != df["dt_quarter"].shift(1)).astype(int)
df["is_quarter_end_trade"]     = (df["dt_quarter"] != df["dt_quarter"].shift(-1)).astype(int)
df["is_month_start_trade"]     = (df["dt_month"] != df["dt_month"].shift(1)).astype(int)
df["is_month_end_trade"]       = (df["dt_month"] != df["dt_month"].shift(-1)).astype(int)
df["is_fortnight_start_trade"] = (df["dt_fortnight"] != df["dt_fortnight"].shift(1)).astype(int)
df["is_fortnight_end_trade"]   = (df["dt_fortnight"] != df["dt_fortnight"].shift(-1)).astype(int)
df["is_week_start_trade"]      = (df["dt_week"] != df["dt_week"].shift(1)).astype(int)
df["is_week_end_trade"]        = (df["dt_week"] != df["dt_week"].shift(-1)).astype(int)
df["is_year_start_order"]      = df["is_year_start_trade"].shift(-1, fill_value=0)
df["is_year_end_order"]        = df["is_year_end_trade"].shift(-1, fill_value=0)
df["is_quarter_start_order"]   = df["is_quarter_start_trade"].shift(-1, fill_value=0)
df["is_quarter_end_order"]     = df["is_quarter_end_trade"].shift(-1, fill_value=0)
df["is_month_start_order"]     = df["is_month_start_trade"].shift(-1, fill_value=0)
df["is_month_end_order"]       = df["is_month_end_trade"].shift(-1, fill_value=0)
df["is_fortnight_start_order"] = df["is_fortnight_start_trade"].shift(-1, fill_value=0)
df["is_fortnight_end_order"]   = df["is_fortnight_end_trade"].shift(-1, fill_value=0)
df["is_week_start_order"]      = df["is_week_start_trade"].shift(-1, fill_value=0)
df["is_week_end_order"]        = df["is_week_end_trade"].shift(-1, fill_value=0)

df["day_of_year_trade"]      = df.groupby("dt_year").cumcount() + 1
df["day_of_quarter_trade"]   = df.groupby("dt_quarter").cumcount() + 1
df["day_of_month_trade"]     = df.groupby("dt_month").cumcount() + 1
df["day_of_fortnight_trade"] = df.groupby("dt_fortnight").cumcount() + 1
df["day_of_week_trade"]      = df.groupby("dt_week").cumcount() + 1

df["day_of_year_order"]      = df["day_of_year_trade"].shift(-1, fill_value=0)
df["day_of_quarter_order"]   = df["day_of_quarter_trade"].shift(-1, fill_value=0)
df["day_of_month_order"]     = df["day_of_month_trade"].shift(-1, fill_value=0)
df["day_of_fortnight_order"] = df["day_of_fortnight_trade"].shift(-1, fill_value=0)
df["day_of_week_order"]      = df["day_of_week_trade"].shift(-1, fill_value=0)

In [17]:
gap_from_prev = (df["date"] - df["date"].shift(1)).dt.days
gap_to_next   = (df["date"].shift(-1) - df["date"]).dt.days

df["is_holiday_begin_trade"] = (gap_to_next > 1).fillna(False).astype(int)
df["is_holiday_end_trade"]   = (gap_from_prev > 1).fillna(False).astype(int)

df["is_holiday_begin_order"] = df["is_holiday_begin_trade"].shift(-1, fill_value=0)
df["is_holiday_end_order"]   = df["is_holiday_end_trade"].shift(-1, fill_value=0)

In [18]:
df = df[1:]
df["is_holiday_end_order"].iloc[-1] = 1
df["is_holiday_begin_order"].iloc[-2] = 1
df

,date,dt_year,dt_quarter,dt_month,dt_fortnight,dt_week,dt_day,is_year_start_trade,is_year_end_trade,is_quarter_start_trade,...,day_of_week_trade,day_of_year_order,day_of_quarter_order,day_of_month_order,day_of_fortnight_order,day_of_week_order,is_holiday_begin_trade,is_holiday_end_trade,is_holiday_begin_order,is_holiday_end_order
1,1990-12-04,1990,199004,199012,199025,199048,19901204,0,0,0,...,2,3,3,3,3,3,0,0,0,0
2,1990-12-05,1990,199004,199012,199025,199048,19901205,0,0,0,...,3,4,4,4,4,4,0,0,0,0
3,1990-12-06,1990,199004,199012,199025,199048,19901206,0,0,0,...,4,5,5,5,5,5,0,0,1,0
4,1990-12-07,1990,199004,199012,199025,199048,19901207,0,0,0,...,5,6,6,6,6,1,1,0,0,1
5,1990-12-10,1990,199004,199012,199025,199049,19901210,0,0,0,...,1,7,7,7,7,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8804,2026-12-25,2026,202604,202612,202626,202651,20261225,0,0,0,...,5,239,58,20,8,1,1,0,0,1
8805,2026-12-28,2026,202604,202612,202626,202652,20261228,0,0,0,...,1,240,59,21,9,2,0,1,0,0
8806,2026-12-29,2026,202604,202612,202626,202652,20261229,0,0,0,...,2,241,60,22,10,3,0,0,0,0
8807,2026-12-30,2026,202604,202612,202626,202652,20261230,0,0,0,...,3,242,61,23,1,4,0,0,1,0


In [19]:
# Save to DAI
ds = dai.DataSource("mldt_cn_stock_calendar_daily")
def update_df(df_new):
    df_new = df
    return df_new
ds.apply_bdb(update_df, as_type=pd.DataFrame)

dai.DataSource.write_bdb(
    data=df,
    id="mldt_cn_stock_calendar_daily",
    unique_together=["date"],
    indexes=["date"],
)

dai.DataSource("mldt_cn_stock_calendar_daily")